In [2]:
import re
import shlex

# moleulce = "CLN025" #chignolin
# molecule = "2JOF" # trp-cage
molecule = "2F4K" # villin
simulation_idx = "0"

In [3]:
mae_filepath = f"/home/shpark/prj-mlcv/lib/DESRES/DESRES-Trajectory_{molecule}-{simulation_idx}-protein/system.mae"
with open(mae_filepath, 'r') as file:
	mae_content = file.read()

# Find the atom block using regex
atom_block_match = re.search(r'm_atom\[\d+\]\s*{([^}]*)}', mae_content, re.DOTALL)

if not atom_block_match:
	raise ValueError("Atom block not found in the MAE file.")

atom_block = atom_block_match.group(1).strip()
atom_lines = atom_block.split('\n')


# Extract header fields (lines before ':::')
header_fields = []
coords_started = False
atom_data = []

for line in atom_lines:
	line = line.strip()
	if line == ':::':
		coords_started = True
		continue
	
	if "TIP3" in line:
		coords_started = False
		break

	if not coords_started:
		header_fields.extend(line.split())
	else:
		atom_data.append(line)

# Map field names to their indices
header_indices = {name: idx for idx, name in enumerate(header_fields)}

In [4]:
print(len(atom_data))
print(atom_data[0])
print(atom_data[1])
print(atom_data[2])

# Example
# 1 " N  " TYR P P 1 10.865962 -13.578603 -19.419243 7 1 43 1 1.6651826 -0.85347438 1.6885613
# 2 " HT1" TYR P P 1 11.192533 -12.732378 -19.863485 1 1 21 1 2.0433135 -9.0181837 -13.586321

577
1 " N  " LEU X "" 42 -3.6757333 -13.751919 3.9497461 -1.2858058 -3.765785 2.9075887 7 1 43 1
2 " CA " LEU X "" 42 -3.8230925 -14.504919 5.1855025 4.1888099 3.3171127 -7.4906654 6 1 2 1
3 " CB " LEU X "" 42 -2.5766435 -14.529973 6.1022749 -1.290229 -7.5284152 -5.5498562 6 1 2 1


In [49]:
def infer_element(atom_name, atomic_number=None):
    """
    Infer element symbol from atom name or fallback to atomic number.
    """
    atom_name = atom_name.strip().upper()

    if atom_name.startswith("H"):
        return "H"
    elif atom_name.startswith("C"):
        return "C"
    elif atom_name.startswith("N"):
        return "N"
    elif atom_name.startswith("O"):
        return "O"
    elif atom_name.startswith("S"):
        return "S"
    elif atomic_number is not None:
        # fallback using periodic table
        periodic_table = {1: "H", 6: "C", 7: "N", 8: "O", 16: "S"}
        return periodic_table.get(atomic_number, "X")
    else:
        return "X"

pdb_lines = []
for line in atom_data:
    tokens = shlex.split(line)
    
    atom_idx = int(tokens[0])
    atom_name = tokens[1].replace('"', '')
    residue_name = tokens[2]
    residue_number = int(tokens[5])
    
    x = f"{float(tokens[6]):.3f}"
    y = f"{float(tokens[7]):.3f}"
    z = f"{float(tokens[8]):.3f}"
    element = infer_element(atom_name)

    # pdb_line = f"ATOM {atom_idx} {atom_name} {residue_name} {residue_number} {x} {y} {z}  1.00  0.00 {element}"
    pdb_line = "ATOM {:>6} {:<4} {:>3} {:>5} {:>11} {:>7} {:>7}  1.00  0.00{:>12}".format(
        atom_idx,
        atom_name,
        residue_name,
        residue_number - 41,
        x,
        y,
        z,
        element
	)
    print(pdb_line)
    pdb_lines.append(pdb_line)
pdb_lines.append("END")

ATOM      1  N   LEU     1      -3.676 -13.752   3.950  1.00  0.00           N
ATOM      2  CA  LEU     1      -3.823 -14.505   5.186  1.00  0.00           C
ATOM      3  CB  LEU     1      -2.577 -14.530   6.102  1.00  0.00           C
ATOM      4  CG  LEU     1      -1.193 -15.031   5.538  1.00  0.00           C
ATOM      5  CD1 LEU     1       0.039 -14.849   6.402  1.00  0.00           C
ATOM      6  CD2 LEU     1      -1.224 -16.532   5.195  1.00  0.00           C
ATOM      7  C   LEU     1      -5.087 -14.004   5.956  1.00  0.00           C
ATOM      8  O   LEU     1      -6.010 -14.780   6.124  1.00  0.00           O
ATOM      9  N   SER     2      -5.106 -12.748   6.408  1.00  0.00           N
ATOM     10  CA  SER     2      -6.249 -12.072   6.997  1.00  0.00           C
ATOM     11  C   SER     2      -6.311 -10.618   6.527  1.00  0.00           C
ATOM     12  O   SER     2      -5.360 -10.122   5.887  1.00  0.00           O
ATOM     13  CB  SER     2      -6.169 -12.031   8.5

In [46]:
# Sort pdb_lines by residue number
# First, extract residue numbers from the existing pdb_lines (excluding the "END" line)
pdb_data_with_residue = []

for line in pdb_lines[:-1]:  # Exclude the "END" line
    # Extract residue number from the PDB line (positions 22-26 in PDB format)
    residue_num = int(line[22:27].strip())
    pdb_data_with_residue.append((residue_num, line))

# Sort by residue number
pdb_data_with_residue.sort(key=lambda x: x[0])

# Rebuild pdb_lines with sorted data
pdb_lines = [line for _, line in pdb_data_with_residue]
pdb_lines.append("END")

# print("PDB lines sorted by residue number:")
# for line in pdb_lines[:10]:  # Print first 10 lines as sample
#     print(line)
    
pdb_lines


['ATOM      1  N   LEU     1      -3.676 -13.752   3.950  1.00  0.00           N',
 'ATOM      2  CA  LEU     1      -3.823 -14.505   5.186  1.00  0.00           C',
 'ATOM      3  CB  LEU     1      -2.577 -14.530   6.102  1.00  0.00           C',
 'ATOM      4  CG  LEU     1      -1.193 -15.031   5.538  1.00  0.00           C',
 'ATOM      5  CD1 LEU     1       0.039 -14.849   6.402  1.00  0.00           C',
 'ATOM      6  CD2 LEU     1      -1.224 -16.532   5.195  1.00  0.00           C',
 'ATOM      7  C   LEU     1      -5.087 -14.004   5.956  1.00  0.00           C',
 'ATOM      8  O   LEU     1      -6.010 -14.780   6.124  1.00  0.00           O',
 'ATOM    288  1H  LEU     1      -4.558 -13.729   3.459  1.00  0.00           X',
 'ATOM    289  2H  LEU     1      -3.480 -12.801   4.228  1.00  0.00           X',
 'ATOM    290  3H  LEU     1      -2.933 -14.090   3.355  1.00  0.00           X',
 'ATOM    291  HA  LEU     1      -3.985 -15.573   5.040  1.00  0.00           H',
 'AT

In [47]:
# Renumber atom indices from 1 to 577
pdb_lines_renumbered = []

for i, line in enumerate(pdb_lines[:-1], 1):  # Exclude the "END" line
    # Replace the atom index (positions 6-11 in PDB format) with sequential numbering
    new_line = line[:5] + f"{i:>6} " + line[12:]
    pdb_lines_renumbered.append(new_line)

pdb_lines_renumbered.append("END")

# Update pdb_lines with renumbered version
pdb_lines = pdb_lines_renumbered

# print("First 10 lines with renumbered atom indices:")
# for line in pdb_lines[:10]:
#     print(line)
    
pdb_lines

['ATOM      1  N   LEU     1      -3.676 -13.752   3.950  1.00  0.00           N',
 'ATOM      2  CA  LEU     1      -3.823 -14.505   5.186  1.00  0.00           C',
 'ATOM      3  CB  LEU     1      -2.577 -14.530   6.102  1.00  0.00           C',
 'ATOM      4  CG  LEU     1      -1.193 -15.031   5.538  1.00  0.00           C',
 'ATOM      5  CD1 LEU     1       0.039 -14.849   6.402  1.00  0.00           C',
 'ATOM      6  CD2 LEU     1      -1.224 -16.532   5.195  1.00  0.00           C',
 'ATOM      7  C   LEU     1      -5.087 -14.004   5.956  1.00  0.00           C',
 'ATOM      8  O   LEU     1      -6.010 -14.780   6.124  1.00  0.00           O',
 'ATOM      9  1H  LEU     1      -4.558 -13.729   3.459  1.00  0.00           X',
 'ATOM     10  2H  LEU     1      -3.480 -12.801   4.228  1.00  0.00           X',
 'ATOM     11  3H  LEU     1      -2.933 -14.090   3.355  1.00  0.00           X',
 'ATOM     12  HA  LEU     1      -3.985 -15.573   5.040  1.00  0.00           H',
 'AT

In [50]:
with open(f"{molecule}_from_mae4.pdb", "w") as f:
    f.write("\n".join(pdb_lines))